# Avance RecSys

Importar librerias

In [ ]:
import numpy as np
import torch
from pkg_resources import packaging
from PIL import Image

print("Torch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"

## CLIP

Importar librerias

In [6]:
import Clip.clip.clip as clip

Torch version: 1.13.0+cu117


In [7]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [8]:
model, preprocess = clip.load("ViT-B/32", device=device)
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


Calcular embeddings para el set de test

In [52]:
def evaluate_image(path_img, text_inputs, show=False, top=True):
    im = Image.open(path_img)
    if show:
        im.show()
    image_input = preprocess(im).unsqueeze(0).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)
    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    if top:
        value, index = similarity[0].topk(1)
        return image_features, text_descriptors[index]
    else:
        values, indices = similarity[0].topk(5)
        
        print("\nTop predictions:\n")
        for value, index in zip(values, indices):
            print(f"{text_descriptors[index]:>16s}: {100 * value.item():.2f}%")

In [56]:
def evaluate_test():
    with open("classes.txt", "r") as file:
        tmp = file.readlines()
        text_descriptors = [line.strip() for line in tmp]

    text_inputs = torch.cat([clip.tokenize(c) for c in text_descriptors]).to(device)
    
    with open("images_test.txt", "r") as file:
        tmp = file.readlines()
        images = [line.strip() for line in tmp] 
    
    embeddings = []
    for image in images:
        path = f"iui_wikimedia_images/{image}"
        embedding, result = evaluate_image(path, text_inputs)
        embeddings.append(embedding)
    return embeddings

In [57]:
embeddings = evaluate_test()

## CuratorNet